In [1]:
!pip install delta-spark==3.2.0 -q
import pyspark
from delta import *
from pyspark.sql.functions import *

# Create a SparkSession with Delta Lake extensions
# The '.config(...)' lines are crucial for enabling Delta Lake's features
builder = pyspark.sql.SparkSession.builder.appName("DeltaTutorial") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Get or create the SparkSession
spark = configure_spark_with_delta_pip(builder).getOrCreate()

print("Spark and Delta Lake are ready!")

Spark and Delta Lake are ready!


In [2]:
#Managed data - if DB got deleted whole data and metadata also got deleted.
#Unmanaged data - if DB got deleted only metadata will gone data will still present.

data = [("Amit", 28), ("Priya", 32), ("Rahul", 25)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.write.format("delta").saveAsTable("managed_people")

spark.sql("SELECT * FROM managed_people").show()

location = spark.sql("DESCRIBE DETAIL managed_people").collect()[0]["location"]
print("Managed table location: ", location)

+-----+---+
| Name|Age|
+-----+---+
|Priya| 32|
|Rahul| 25|
| Amit| 28|
+-----+---+

Managed table location:  file:/content/spark-warehouse/managed_people


In [3]:
!pip install pyspark==3.5.0


from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSVExample") \
    .config("spark.sql.catalogImplementation", "in-memory") \
    .getOrCreate()


import pandas as pd
import os

data = {
    "Name": ["Amit", "Priya", "Rahul"],
    "Age": [28, 32, 25]
}
csv_path = "/content/sample_people.csv"
pd.DataFrame(data).to_csv(csv_path, index=False)
print(f"CSV created at: {csv_path}")


df_spark = spark.read.csv(csv_path, header=True, inferSchema=True)
df_spark.show()

if os.path.exists(csv_path):
    os.remove(csv_path)
    print("CSV file deleted successfully.")
else:
    print("CSV file not found.")

CSV created at: /content/sample_people.csv
+-----+---+
| Name|Age|
+-----+---+
| Amit| 28|
|Priya| 32|
|Rahul| 25|
+-----+---+

CSV file deleted successfully.
